In [1]:
#import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [3]:
#configuration data
file_path = "D:/school/dsc425/finalproject/final_data/"
file_name = 'dataset_v1_20201008.csv'
full_path = file_path+file_name

In [4]:
#load the data into a dataframe and set index to date
vix_combined_data = pd.read_csv(full_path)
vix_combined_df_clean = vix_combined_data .loc[:, ~vix_combined_data .columns.str.contains('^Unnamed')] #remove unnamed columns
vix_combined_df_clean.index = pd.to_datetime(vix_combined_df_clean['Date']) #set date to index
del vix_combined_df_clean['Date'] # delete the date column as now index



In [ ]:
vix_combined_df_clean

In [5]:
#clean out the ln columns where ln was hard coded from the original file - recalculating using python
for col in vix_combined_df_clean.columns:
    if  'ln' in col or 'RV' in col:
        del vix_combined_df_clean[col] 

In [6]:
# create a log for each spx column
for col in vix_combined_df_clean.columns:
    if 'spx' in col:
        new_column_name = "ln_"+ col
        vix_combined_df_clean[new_column_name] = np.log(vix_combined_df_clean[col])

In [7]:
#create  50 day and 200 day moving average values
column_list = list(vix_combined_df_clean.columns)
for col in column_list:
    if 'spx' in col:
        ma_50_day_col_name = col +"_" + "50_day_MA"
        ma_200_day_col_name = col +"_" + "200_day_MA"
        vix_combined_df_clean[ma_50_day_col_name] = vix_combined_df_clean[col].rolling(50,min_periods=1).mean()



In [ ]:
print(vix_combined_df_clean)

In [8]:
vix_combined_df_clean.to_csv(file_path+'vix_panda_with_index.csv',index=True)

In [ ]:
x_axis = vix_combined_data.index
x_axis_name = 'Date'
for col in vix_combined_df_clean.columns:
    if  'spx' in col:
        y1 = vix_combined_df_clean[col]
        y1_name = col
        for col2 in vix_combined_df_clean.columns:
            if 'vix' in col2:
                y2 = vix_combined_df_clean[col2]
                y2_name = col2
                data_columns = [y1,y2]
                data_column_names = [x_axis_name,y1_name,y2_name]
                print(col, col2)
                create_dual_plots(x_axis, data_columns,data_column_names)

In [ ]:
vix_test = pd.read_csv('vix_panda_with_index.csv')

In [ ]:
def create_dual_plots(x_axis, data_columns, data_column_names):
    '''
        Take in as input x-axis two lists column data and column name
    '''
    fig, ax = plt.subplots(figsize=(10, 6))
    axb = ax.twinx()
    df=vix_combined_data
    x = x_axis
    y1 = data_columns[0]
    y2 = data_columns[1]
    x_axis_name = data_column_names[0]
    y1_column_name = data_column_names[1]
    y2_column_name = data_column_names[2]
    title_temp = y1_column_name + " " + y2_column_name
    # Same as above
    ax.set_xlabel(x_axis_name)
    ax.set_ylabel(y1_column_name)
    ax.set_title(title_temp)
    ax.grid(True)

    # Plotting on the first y-axis
    ax.plot(x, y1, color='tab:orange', label=y1_column_name)
    ax.legend(loc='upper left')

    # Plotting on the second y-axis
    axb.set_ylabel(y2_column_name)
    axb.plot(x, y2, color='black', label=y2_column_name)
    axb.legend(loc='upper right')

    path = "D:/school/dsc425/finalproject/graphs/"
    file_name = path+title_temp + ".png"
    plt.savefig(file_name)
